# Candidate Evaluation with AISuite
> format: ipynb

> jupyter: python3

# Candidate Evaluation with AISuite

This notebook evaluates job candidates using AISuite by combining system and user prompts 
stored in external files. It retrieves credentials, loads necessary data, 
constructs prompts, and generates AI-driven candidate evaluations.

In [ ]:
import os
import json
import pandas as pd
from pathlib import Path
import aisuite as ai
from IPython.display import display, Markdown

## Configuration and Setup
Define file paths and load credentials.

In [ ]:
# Paths configuration
paths = {
    'root': Path.cwd().parent,
    'data': Path.cwd().parent / "data",
    "config": Path.cwd().parent / "config"
}

# Load credentials from config/credentials.json
with open(paths["config"] / 'credentials.json') as f:
    credentials = json.load(f)

# Set necessary environment variables for API tokens
os.environ["HUGGINGFACE_TOKEN"] = credentials.get("HUGGINGFACE_TOKEN", "")
os.environ["GROQ_API_KEY"] = credentials.get("GROQ_API_KEY", "")
os.environ["OPENAI_API_KEY"] = credentials.get("OPENAI_API_KEY", "")

if os.environ["HUGGINGFACE_TOKEN"]:
    print("Environment variable HUGGINGFACE_TOKEN set.")

Environment variable HUGGINGFACE_TOKEN set.


## Load External Text Prompts
Load system description and user prompt template from external files.

In [ ]:
# Load system description and user prompt template from files
with open(paths["config"] / 'role_system_description.txt', 'r') as f:
    role_system_description = f.read()

with open(paths["config"] / 'user_prompt_template.txt', 'r') as f:
    user_prompt_template = f.read()

## Load Instructions and Candidate Data

In [ ]:
# Load instructions (if needed for context)
try:
    with open(paths["config"] / "instructions.txt", 'r') as file:
        instructions = file.read()
        print("Instructions successfully read!")
except FileNotFoundError:
    instructions = ""
    print("Instructions file not found.")

# Load shortlisted candidates from parquet or CSV
try:
    list_of_candidates = (
        pd.read_parquet(paths['data'] / "processed/filtered.parquet", columns=['job_title', 'rank'])
          .set_index("rank")['job_title']
          .to_list()
    )
except Exception as e:
    print(f"Failed to load parquet file: {e}. Loading CSV instead.")
    list_of_candidates = (
        pd.read_csv(paths['data'] / "processed/filtered.csv", index_col=['rank'], usecols=['job_title'])
          ['job_title']
          .to_list()
    )


Instructions successfully read!


## Prepare Dynamic Prompt Parameters

In [ ]:
# Define dynamic parameters for the prompt
list_similar_roles = 'aspiring human resources'
location = "New York"

search_criteria = """**Criteria:**
- **Motivation:** Demonstrated interest and passion for a career in Human Resources.
- **Experience & Background:** Relevant experience, transferable skills, and educational background.
- **Fit with Company Values:** Highly motivated individuals with fundamental HR knowledge are preferred over those with extensive credentials.
"""

# Format the user prompt using the template and parameters
user_prompt = user_prompt_template.format(
    location=location,
    list_similar_roles=list_similar_roles,
    search_criteria=search_criteria,
    list_of_candidates="\n".join(list_of_candidates)  # Joining list elements for display
)

# Combine system description and user prompt for models that don't support system roles
combined_prompt = f"{role_system_description}\n\n{user_prompt}"

## Define AISuite Generation Function

In [ ]:
def aisuite_generate(messages, model="openai:o1-mini", max_tokens=1500, temperature=0.0):
    """
    Generate a response using AISuite's chat completion API.
    
    Args:
        messages (list): List of message dicts for the model.
        model (str): Model identifier to use.
        max_tokens (int): Maximum number of tokens to generate.
        temperature (float): Sampling temperature.

    Returns:
        str: Generated response text.
    """
    client = ai.Client(
        provider_configs={
            "groq": {
                "api_key": os.environ["GROQ_API_KEY"]
            },
            # Add additional provider configurations as required
        }
    )
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_completion_tokens=max_tokens,
        # temperature=temperature,  # Uncomment if supported
    )
    return response.choices[0].message.content

## Generate and Display Candidate Evaluation

In [ ]:
# Prepare messages
messages = [{"role": "user", "content": combined_prompt}]

# Generate text using AISuite
generated_output = aisuite_generate(messages, model="openai:o1-mini", max_tokens=5000)

# Display the generated output in Markdown format
display(Markdown("**Generated Output:**\n" + generated_output))

**Generated Output:**
Based on the provided search criteria and location preference for **New York**, the following table ranks the top 5 candidates most suitable for an aspiring Human Resources position. The evaluation prioritizes motivation, relevant experience and background, and alignment with company values emphasizing fundamental HR knowledge and enthusiasm over extensive credentials.

| **Candidate**                                                                 | **Experience & Background**                                                                                   | **Reasoning for Choosing This Candidate**                                                                                                                                                   |
|-------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Aspiring Human Resources Professional - Energetic and Team-Focused Leader** | Demonstrates strong leadership skills and teamwork experience, indicating potential for HR roles that require collaboration and employee engagement. | Exhibits high motivation and passion for HR through leadership and team-focused attributes, aligning well with company values that prioritize an inclusive and dynamic work environment.       |
| **Aspiring Human Resources Professional - Passionate About Inclusivity**     | Focused on creating an inclusive and engaging work environment, showcasing a commitment to diversity and employee well-being. | Shows a clear passion for fostering an inclusive workplace, which is a key company value. Fundamental HR knowledge is evident through their dedication to enhancing workplace culture.       |
| **CT Bauer College of Business Graduate - Magna Cum Laude**                  | Holds a business degree with honors, providing a solid educational foundation relevant to HR principles and practices. | Combines strong academic performance with an aspiration in HR, indicating both capability and motivation. The educational background ensures fundamental HR knowledge is well-established. |
| **Business Management Major - Aspiring Human Resources Manager**             | Studying business management with a focus on HR management, offering relevant academic and potential practical experience. | Demonstrates a clear career focus in HR management, aligning with the role's requirements. Educational pursuits in business management support a solid understanding of HR fundamentals.     |
| **Experienced Retail Manager - Aspiring Human Resources Professional**       | Brings substantial management experience from the retail sector, highlighting transferable skills such as employee supervision and conflict resolution. | Leverages extensive experience in a managerial role to transition into HR, showing motivation to apply transferable skills. This practical background complements fundamental HR knowledge.    |